# Import Dependencies 

In [1]:
!pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl

Processing /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl


In [2]:
 # Importing Required Libraries\nLet's begin by importing the essential Python libraries needed for data processing, visualization, and modeling.

import pandas as pd
import numpy as np


from sklearn.ensemble import HistGradientBoostingRegressor,ExtraTreesRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


import networkx as nx
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdmolops
from rdkit import Chem

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [3]:
class CFG:
    TARGETS = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
    SEED = 42
    FOLDS = 5

In [4]:
useless_cols = [   
    
    'MaxPartialCharge', 
    # Nan data
    'BCUT2D_MWHI',
    'BCUT2D_MWLOW',
    'BCUT2D_CHGHI',
    'BCUT2D_CHGLO',
    'BCUT2D_LOGPHI',
    'BCUT2D_LOGPLOW',
    'BCUT2D_MRHI',
    'BCUT2D_MRLOW',

    # Constant data
    'NumRadicalElectrons',
    'SMR_VSA8',
    'SlogP_VSA9',
    'fr_barbitur',
    'fr_benzodiazepine',
    'fr_dihydropyridine',
    'fr_epoxide',
    'fr_isothiocyan',
    'fr_lactam',
    'fr_nitroso',
    'fr_prisulfonamd',
    'fr_thiocyan',

    # High correlated data >0.95
    'MaxEStateIndex',
    'HeavyAtomMolWt',
    'ExactMolWt',
    'NumValenceElectrons',
    'Chi0',
    'Chi0n',
    'Chi0v',
    'Chi1',
    'Chi1n',
    'Chi1v',
    'Chi2n',
    'Kappa1',
    'LabuteASA',
    'HeavyAtomCount',
    'MolMR',
    'Chi3n',
    'BertzCT',
    'Chi2v',
    'Chi4n',
    'HallKierAlpha',
    'Chi3v',
    'Chi4v',
    'MinAbsPartialCharge',
    'MinPartialCharge',
    'MaxAbsPartialCharge',
    'FpDensityMorgan2',
    'FpDensityMorgan3',
    'Phi',
    'Kappa3',
    'fr_nitrile',
    'SlogP_VSA6',
    'NumAromaticCarbocycles',
    'NumAromaticRings',
    'fr_benzene',
    'VSA_EState6',
    'NOCount',
    'fr_C_O',
    'fr_C_O_noCOO',
    'NumHDonors',
    'fr_amide',
    'fr_Nhpyrrole',
    'fr_phenol',
    'fr_phenol_noOrthoHbond',
    'fr_COO2',
    'fr_halogen',
    'fr_diazo',
    'fr_nitro_arom',
    'fr_phos_ester'
]


# Read Files

### Main Files

In [5]:
 #We will load both the training and test datasets using pandas, and store test IDs 
train=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
test=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')
ss=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv')
ID=test['id'].copy()

### Extra Files

In [6]:
tc_smiles =pd.read_csv('/kaggle/input/tc-smiles/Tc_SMILES.csv')
tg_smiles =pd.read_csv('/kaggle/input/smiles-extra-data/JCIM_sup_bigsmiles.csv')
ktg_smiles =pd.read_excel('/kaggle/input/smiles-extra-data/data_tg3.xlsx')
de_smiles =pd.read_excel('/kaggle/input/smiles-extra-data/data_dnst1.xlsx')
ffv_smiles =pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv')

# Preprocessing 

In [7]:
def make_smile_canonical(smile): # To avoid duplicates, for example: canonical '*C=C(*)C' == '*C(=C*)C'
    try:
        mol = Chem.MolFromSmiles(smile)
        canon_smile = Chem.MolToSmiles(mol, canonical=True)
        return canon_smile
    except:
        return np.nan

train['SMILES'] = train['SMILES'].apply(lambda s: make_smile_canonical(s))
test['SMILES'] = test['SMILES'].apply(lambda s: make_smile_canonical(s))

In [8]:
ktg_smiles.rename(columns={'Tg [K]': 'Tg'}, inplace=True)
tg_smiles.rename(columns={'Tg (C)': 'Tg'}, inplace=True)
tc_smiles.rename(columns={'TC_mean': 'Tc'}, inplace=True)
de_smiles.rename(columns={'density(g/cm3)': 'Density'}, inplace=True)
ffv_smiles.rename(columns={'FFV': 'FFV'}, inplace=True)

In [9]:
de_smiles['SMILES'] = de_smiles['SMILES'].apply(lambda s: make_smile_canonical(s))
de_smiles = de_smiles[(de_smiles['SMILES'].notnull())&(de_smiles['Density'].notnull())&(de_smiles['Density'] != 'nylon')]
de_smiles['Density'] = de_smiles['Density'].astype('float64')
de_smiles['Density'] -= 0.118

ktg_smiles['Tg'] = ktg_smiles['Tg'] - 273.15

[10:22:22] SMILES Parse Error: syntax error while parsing: *O[Si](*)([R])[R]
[10:22:22] SMILES Parse Error: check for mistakes around position 12:
[10:22:22] *O[Si](*)([R])[R]
[10:22:22] ~~~~~~~~~~~^
[10:22:22] SMILES Parse Error: Failed parsing SMILES '*O[Si](*)([R])[R]' for input: '*O[Si](*)([R])[R]'
[10:22:22] SMILES Parse Error: syntax error while parsing: *NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4
[10:22:22] SMILES Parse Error: check for mistakes around position 28:
[10:22:22] c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=
[10:22:22] ~~~~~~~~~~~~~~~~~~~~^
[10:22:22] SMILES Parse Error: Failed parsing SMILES '*NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4' for input: '*NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4'
[10:22:22] SMILES Parse Error: syntax error while parsing: O=C=N[R1]N=C=O.O[R2]O.O[R3]O
[10:22:22] SMILES Parse Error: check for mistakes around position 7:
[10:22:22] O=C=N[R1]N=C=O.O[R2]O.O[R3]O
[10:22:22] ~~~~~~^
[10:22:22] SMILES Parse Error: F

In [10]:
def preprocessing(df):
    desc_names = [desc[0] for desc in Descriptors.descList if desc[0] not in useless_cols]
    descriptors = [compute_all_descriptors(smi) for smi in df['SMILES'].to_list()]

    graph_feats = {'graph_diameter': [], 'avg_shortest_path': [], 'num_cycles': []}
    for smile in df['SMILES']:
         compute_graph_features(smile, graph_feats)
        
    result = pd.concat(
        [
            pd.DataFrame(descriptors, columns=desc_names),
            pd.DataFrame(graph_feats)
        ],
        axis=1
    )

    result = result.replace([-np.inf, np.inf], np.nan)
    return result

In [11]:
train

,id,SMILES,Tg,FFV,Tc,Density,Rg
0,87817,*CC(*)c1ccccc1C(=O)OCCCCCC,NaN,0.374645,0.205667,NaN,NaN
1,106919,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,NaN,0.370410,NaN,NaN,NaN
2,388772,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,NaN,0.378860,NaN,NaN,NaN
3,519416,*Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)...,NaN,0.387324,NaN,NaN,NaN
4,539187,*Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N...,NaN,0.355470,NaN,NaN,NaN
...,...,...,...,...,...,...,...
7968,2146592435,*Oc1cc(CCCCCCCC)cc(OC(=O)c2cccc(C(*)=O)c2)c1,NaN,0.367498,NaN,NaN,NaN
7969,2146810552,*C(=O)OCCN(CCOC(=O)c1ccc2c(c1)C(=O)N(c1cccc(N3...,NaN,0.353280,NaN,NaN,NaN
7970,2147191531,*c1cc(C(=O)NCCCCCCCC)cc(N2C(=O)c3ccc(-c4ccc5c(...,NaN,0.369411,NaN,NaN,NaN
7971,2147435020,*C=C(*)c1ccccc1C,261.662355,NaN,NaN,NaN,NaN


# Feature Extraction 

In [12]:

def add_extra_data(df_train, df_extra, target):
    n_samples_before = len(df_train[df_train[target].notnull()])
    
    df_extra['SMILES'] = df_extra['SMILES'].apply(lambda s: make_smile_canonical(s))
    df_extra = df_extra.groupby('SMILES', as_index=False)[target].mean()
    cross_smiles = set(df_extra['SMILES']) & set(df_train['SMILES'])
    unique_smiles_extra = set(df_extra['SMILES']) - set(df_train['SMILES'])

    # Make priority target value from competition's df
    for smile in df_train[df_train[target].notnull()]['SMILES'].tolist():
        if smile in cross_smiles:
            cross_smiles.remove(smile)

    # Imput missing values for competition's SMILES
    for smile in cross_smiles:
        df_train.loc[df_train['SMILES']==smile, target] = df_extra[df_extra['SMILES']==smile][target].values[0]
    
    df_train = pd.concat([df_train, df_extra[df_extra['SMILES'].isin(unique_smiles_extra)]], axis=0).reset_index(drop=True)

    n_samples_after = len(df_train[df_train[target].notnull()])
    print(f'\nFor target "{target}" added {n_samples_after-n_samples_before} new samples!')
    print(f'New unique SMILES: {len(unique_smiles_extra)}')
    return df_train

train = add_extra_data(train, tc_smiles, 'Tc')
train = add_extra_data(train, tg_smiles, 'Tg')
train = add_extra_data(train, ktg_smiles, 'Tg')
train = add_extra_data(train, de_smiles, 'Density')
train = add_extra_data(train, ffv_smiles, 'FFV')


For target "Tc" added 129 new samples!
New unique SMILES: 129

For target "Tg" added 151 new samples!
New unique SMILES: 136

For target "Tg" added 499 new samples!
New unique SMILES: 499

For target "Density" added 634 new samples!
New unique SMILES: 524

For target "FFV" added 862 new samples!
New unique SMILES: 819


In [13]:
def compute_all_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [None] * len(desc_names)
    return [desc[1](mol) for desc in Descriptors.descList if desc[0] not in useless_cols]

def compute_graph_features(smiles, graph_feats):
    mol = Chem.MolFromSmiles(smiles)
    adj = rdmolops.GetAdjacencyMatrix(mol)
    G = nx.from_numpy_array(adj)

    graph_feats['graph_diameter'].append(nx.diameter(G) if nx.is_connected(G) else 0)
    graph_feats['avg_shortest_path'].append(nx.average_shortest_path_length(G) if nx.is_connected(G) else 0)
    graph_feats['num_cycles'].append(len(list(nx.cycle_basis(G))))

train = pd.concat([train, preprocessing(train)], axis=1)
test = pd.concat([test, preprocessing(test)], axis=1)

# Find constant columns for each target
all_features = train.columns[7:].tolist()
features = {}
for target in CFG.TARGETS:
    const_descs = []
    for col in train.columns.drop(CFG.TARGETS):
        if train[train[target].notnull()][col].nunique() == 1:
            const_descs.append(col)
    features[target] = [f for f in all_features if f not in const_descs]

print(train.shape)
train['Ipc']=np.log10(train['Ipc'])  
for n in train.columns[7:]:
    train[n]=train[n].replace(-np.inf,np.nan)
    train[n]=train[n].replace(np.inf,np.nan)    
    train[n].fillna(train[n].mean())
  
print(train.shape)
test['Ipc']=np.log10(test['Ipc'])
for n in test.columns[7:]:
    train[n]=train[n].replace(-np.inf,np.nan)
    train[n]=train[n].replace(np.inf,np.nan)      
    test[n].fillna(train[n].mean())

(10080, 158)
(10080, 158)


# Data Preparation For Model Training 

In [14]:
# We'll separate train to be one model for each target variable.
t_1=train[['SMILES','Tg']].copy()
t_2=train[['SMILES','FFV']].copy()
t_3=train[['SMILES','Tc']].copy()
t_4=train[['SMILES','Density']].copy()
t_5=train[['SMILES','Rg']].copy()

# We will drop the rows with missing values related to that target after separation.
#This is important , dropping them beforehand would result Null for all data.
t_1.dropna(inplace=True)
t_2.dropna(inplace=True)
t_3.dropna(inplace=True)
t_4.dropna(inplace=True)
t_5.dropna(inplace=True)

In [15]:
train=train.drop(['id','Tg','FFV','Tc','Density','Rg'],axis=1)
test=test.drop(['id','SMILES'],axis=1)

In [16]:
tg=t_1.merge(train,on='SMILES',how='left')
ffv=t_2.merge(train,on='SMILES',how='left')
tc=t_3.merge(train,on='SMILES',how='left')
density=t_4.merge(train,on='SMILES',how='left')
rg=t_5.merge(train,on='SMILES',how='left')

In [17]:
for i in (tg,tc,density,ffv,rg):
    i.drop('SMILES',axis=1,inplace=True)
    i.dropna(inplace=True)

# Model 

In [18]:
train

,SMILES,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,FpDensityMorgan1,AvgIpc,BalabanJ,Ipc,Kappa2,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA9,SlogP_VSA1,SlogP_VSA10,SlogP_VSA11,SlogP_VSA12,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA7,SlogP_VSA8,TPSA,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState10,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState7,VSA_EState8,VSA_EState9,FractionCSP3,NHOHCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAmideBonds,NumAromaticHeterocycles,NumAtomStereoCenters,NumBridgeheadAtoms,NumHAcceptors,NumHeteroatoms,NumHeterocycles,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,NumSpiroAtoms,NumUnspecifiedAtomStereoCenters,RingCount,MolLogP,fr_Al_COO,fr_Al_OH,fr_Al_OH_noTert,fr_ArN,fr_Ar_COO,fr_Ar_N,fr_Ar_NH,fr_Ar_OH,fr_COO,fr_C_S,fr_HOCCN,fr_Imine,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_Ndealkylation1,fr_Ndealkylation2,fr_SH,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amidine,fr_aniline,fr_aryl_methyl,fr_azide,fr_azo,fr_bicyclic,fr_ester,fr_ether,fr_furan,fr_guanido,fr_hdrzine,fr_hdrzone,fr_imidazole,fr_imide,fr_isocyan,fr_ketone,fr_ketone_Topliss,fr_lactone,fr_methoxy,fr_morpholine,fr_nitro,fr_nitro_arom_nonortho,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phos_acid,fr_piperdine,fr_piperzine,fr_priamide,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea,graph_diameter,avg_shortest_path,num_cycles
0,*CC(*)c1ccccc1C(=O)OCCCCCC,12.144536,0.105927,-0.105927,0.500278,13.705882,232.323,1.411765,2.456411,2.563477,4.261290,5.852071,0.000000,0.000000,0.0,0.000000,0.000000,103.451541,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,9.531400,5.969305,0.0,0.000000,0.000000,45.951583,6.606882,35.392371,0.000000,0.000000,0.000000,0.000000,0.000000,12.576187,4.736863,0.000000,54.949285,6.923737,0.000000,26.30,0.000000,4.794537,0.0,5.969305,6.606882,24.825916,25.328832,0.000000,12.132734,19.056471,4.736863,6.120445,0.000000,12.144536,0.000000,2.105648,-0.105927,5.344954,4.148954,0.000000,0.533333,0,0,0,0,0,0,1,0,2,4,0,8,0,0,0,0,1,1,3.98170,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,12,4.736842,1
1,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,3.523412,0.098918,0.098918,0.125364,16.777778,598.919,0.577778,3.135512,1.451540,10.636376,14.715582,0.000000,0.000000,0.0,0.000000,0.000000,222.907896,0.000000,0.000000,0.0,0.0,32.607024,18.759549,0.000000,0.000000,0.000000,11.374773,0.0,0.000000,5.917906,115.071874,11.467335,130.442582,0.000000,11.467335,11.374773,0.000000,0.000000,0.000000,5.414990,5.917906,143.037592,0.000000,0.000000,24.06,0.000000,0.000000,0.0,5.414990,11.835812,5.917906,121.805341,0.000000,0.000000,129.300420,0.000000,0.326442,0.000000,0.000000,0.000000,18.579143,1.794202,15.456662,6.971589,0.000000,0.441860,2,1,0,1,0,0,2,0,2,4,0,16,1,0,1,0,0,5,12.35960,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,22,8.364477,5
2,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,13.714745,0.107441,-3.829434,0.092387,16.301370,1003.207,0.397260,3.440031,0.743739,16.833145,19.899082,9.473726,22.998047,0.0,9.837253,0.000000,222.531582,0.000000,8.417797,0.0,0.0,43.527933,96.764951,5.414990,9.790967,40.947247,43.676159,0.0,0.000000,0.000000,76.363497,0.000000,227.523761,40.246583,25.383483,0.000000,40.246583,0.000000,22.618839,29.884034,0.000000,73.620379,0.000000,12.152040,122.27,19.674506,21.630131,0.0,30.780169,40.246583,67.218863,23.614092,97.061873,60.682977,4

In [19]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import numpy as np

def model_cv(train_d, model_fn, target, n_splits=5):
    X = train_d.drop(columns=[target])
    y = train_d[target].copy()

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    maes = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        m = model_fn()
        m.fit(X_train, y_train)
        y_pred = m.predict(X_val)
        mae = mean_absolute_error(y_val, y_pred)
        maes.append(mae)
        print(f"Fold {fold}: MAE = {mae:.5f}")

    return np.mean(maes), np.std(maes)

# Model Evaluation

In [20]:
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet, HuberRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor

## Tg

In [21]:
models_to_compare = {
    'RandomForest': lambda: RandomForestRegressor(),
    'ExtraTrees': lambda: ExtraTreesRegressor(),
    'GradientBoosting': lambda: GradientBoostingRegressor(),
    'HistGradientBoosting': lambda: HistGradientBoostingRegressor(),
    'XGBoost': lambda: XGBRegressor(verbosity=0),
    'LightGBM': lambda: LGBMRegressor(verbose=-1),
    'CatBoost': lambda: CatBoostRegressor(verbose=0),
    'Ridge': lambda: Ridge(),
    'Lasso': lambda: Lasso(),
    'ElasticNet': lambda: ElasticNet(),
    'Huber': lambda: HuberRegressor(),
    'KNN': lambda: KNeighborsRegressor(),
    'SVR': lambda: SVR(),
    'AdaBoost': lambda: AdaBoostRegressor(),
}

for name, model_cls in models_to_compare.items():
    try:
        mean_mae, std_mae = model_cv(tg, model_cls, 'Tg', n_splits=5)
        print(f'{name:<20}: MAE = {mean_mae:.5f} ± {std_mae:.5f}')
    except Exception as e:
        print(f'{name:<20}: Failed with error: {e}')

Fold 1: MAE = 38.05820
Fold 2: MAE = 36.53949
Fold 3: MAE = 38.02107
Fold 4: MAE = 40.51479
Fold 5: MAE = 39.65148
RandomForest        : MAE = 38.55701 ± 1.38833
Fold 1: MAE = 37.34371
Fold 2: MAE = 36.43748
Fold 3: MAE = 36.43438
Fold 4: MAE = 38.87310
Fold 5: MAE = 40.04951
ExtraTrees          : MAE = 37.82763 ± 1.42391
Fold 1: MAE = 39.33296
Fold 2: MAE = 36.16619
Fold 3: MAE = 39.85635
Fold 4: MAE = 41.46969
Fold 5: MAE = 39.60527
GradientBoosting    : MAE = 39.28609 ± 1.72807
Fold 1: MAE = 39.72937
Fold 2: MAE = 36.70586
Fold 3: MAE = 37.09003
Fold 4: MAE = 38.83727
Fold 5: MAE = 40.16816
HistGradientBoosting: MAE = 38.50614 ± 1.38670
Fold 1: MAE = 38.57506
Fold 2: MAE = 37.16471
Fold 3: MAE = 42.03334
Fold 4: MAE = 38.59497
Fold 5: MAE = 42.81511
XGBoost             : MAE = 39.83664 ± 2.18949
Fold 1: MAE = 37.76024
Fold 2: MAE = 36.88961
Fold 3: MAE = 37.89726
Fold 4: MAE = 38.35549
Fold 5: MAE = 41.72934
LightGBM            : MAE = 38.52639 ± 1.67033
Fold 1: MAE = 37.35201
Fold 

## FFV

In [22]:
for name, model_cls in models_to_compare.items():
    try:
        mean_mae, std_mae = model_cv(ffv, model_cls, 'FFV', n_splits=5)
        print(f'{name:<20}: MAE = {mean_mae:.5f} ± {std_mae:.5f}')
    except Exception as e:
        print(f'{name:<20}: Failed with error: {e}')

Fold 1: MAE = 0.00701
Fold 2: MAE = 0.00689
Fold 3: MAE = 0.00701
Fold 4: MAE = 0.00693
Fold 5: MAE = 0.00703
RandomForest        : MAE = 0.00697 ± 0.00006
Fold 1: MAE = 0.00616
Fold 2: MAE = 0.00610
Fold 3: MAE = 0.00602
Fold 4: MAE = 0.00597
Fold 5: MAE = 0.00646
ExtraTrees          : MAE = 0.00614 ± 0.00017
Fold 1: MAE = 0.00925
Fold 2: MAE = 0.00952
Fold 3: MAE = 0.00931
Fold 4: MAE = 0.00903
Fold 5: MAE = 0.00951
GradientBoosting    : MAE = 0.00932 ± 0.00018
Fold 1: MAE = 0.00722
Fold 2: MAE = 0.00720
Fold 3: MAE = 0.00714
Fold 4: MAE = 0.00709
Fold 5: MAE = 0.00732
HistGradientBoosting: MAE = 0.00719 ± 0.00008
Fold 1: MAE = 0.00670
Fold 2: MAE = 0.00679
Fold 3: MAE = 0.00682
Fold 4: MAE = 0.00683
Fold 5: MAE = 0.00688
XGBoost             : MAE = 0.00680 ± 0.00006
Fold 1: MAE = 0.00721
Fold 2: MAE = 0.00708
Fold 3: MAE = 0.00715
Fold 4: MAE = 0.00700
Fold 5: MAE = 0.00736
LightGBM            : MAE = 0.00716 ± 0.00012
Fold 1: MAE = 0.00627
Fold 2: MAE = 0.00635
Fold 3: MAE = 0.0064

## Tc

In [23]:
models_to_compare = {
    'RandomForest': lambda: RandomForestRegressor(),
    'ExtraTrees': lambda: ExtraTreesRegressor(),
    'GradientBoosting': lambda: GradientBoostingRegressor(),
    'HistGradientBoosting': lambda: HistGradientBoostingRegressor(),
    'XGBoost': lambda: XGBRegressor(verbosity=0),
    'LightGBM': lambda: LGBMRegressor(verbose=-1),
    'CatBoost': lambda: CatBoostRegressor(verbose=0),
    'Ridge': lambda: Ridge(),
    'Lasso': lambda: Lasso(),
    'ElasticNet': lambda: ElasticNet(),
    'Huber': lambda: HuberRegressor(),
    'KNN': lambda: KNeighborsRegressor(),
    'SVR': lambda: SVR(),
    'AdaBoost': lambda: AdaBoostRegressor(),
}

for name, model_cls in models_to_compare.items():
    try:
        mean_mae, std_mae = model_cv(tc, model_cls, 'Tc', n_splits=5)
        print(f'{name:<20}: MAE = {mean_mae:.5f} ± {std_mae:.5f}')
    except Exception as e:
        print(f'{name:<20}: Failed with error: {e}')

Fold 1: MAE = 0.02987
Fold 2: MAE = 0.03004
Fold 3: MAE = 0.04263
Fold 4: MAE = 0.03097
Fold 5: MAE = 0.03486
RandomForest        : MAE = 0.03367 ± 0.00483
Fold 1: MAE = 0.03099
Fold 2: MAE = 0.02817
Fold 3: MAE = 0.04096
Fold 4: MAE = 0.03314
Fold 5: MAE = 0.03834
ExtraTrees          : MAE = 0.03432 ± 0.00470
Fold 1: MAE = 0.02998
Fold 2: MAE = 0.03137
Fold 3: MAE = 0.04237
Fold 4: MAE = 0.03230
Fold 5: MAE = 0.03541
GradientBoosting    : MAE = 0.03429 ± 0.00442
Fold 1: MAE = 0.02890
Fold 2: MAE = 0.03025
Fold 3: MAE = 0.04082
Fold 4: MAE = 0.03070
Fold 5: MAE = 0.03310
HistGradientBoosting: MAE = 0.03275 ± 0.00425
Fold 1: MAE = 0.02950
Fold 2: MAE = 0.03192
Fold 3: MAE = 0.04290
Fold 4: MAE = 0.03084
Fold 5: MAE = 0.03730
XGBoost             : MAE = 0.03449 ± 0.00497
Fold 1: MAE = 0.02897
Fold 2: MAE = 0.02952
Fold 3: MAE = 0.04185
Fold 4: MAE = 0.03065
Fold 5: MAE = 0.03356
LightGBM            : MAE = 0.03291 ± 0.00474
Fold 1: MAE = 0.02858
Fold 2: MAE = 0.02848
Fold 3: MAE = 0.0398

## Density

In [24]:
models_to_compare = {
    'RandomForest': lambda: RandomForestRegressor(),
    'ExtraTrees': lambda: ExtraTreesRegressor(),
    'GradientBoosting': lambda: GradientBoostingRegressor(),
    'HistGradientBoosting': lambda: HistGradientBoostingRegressor(),
    'XGBoost': lambda: XGBRegressor(verbosity=0),
    'LightGBM': lambda: LGBMRegressor(verbose=-1),
    'CatBoost': lambda: CatBoostRegressor(verbose=0),
    'Ridge': lambda: Ridge(),
    'Lasso': lambda: Lasso(),
    'ElasticNet': lambda: ElasticNet(),
    'Huber': lambda: HuberRegressor(),
    'KNN': lambda: KNeighborsRegressor(),
    'SVR': lambda: SVR(),
    'AdaBoost': lambda: AdaBoostRegressor(),
}

for name, model_cls in models_to_compare.items():
    try:
        mean_mae, std_mae = model_cv(density, model_cls, 'Density', n_splits=5)
        print(f'{name:<20}: MAE = {mean_mae:.5f} ± {std_mae:.5f}')
    except Exception as e:
        print(f'{name:<20}: Failed with error: {e}')

Fold 1: MAE = 0.03897
Fold 2: MAE = 0.04464
Fold 3: MAE = 0.04289
Fold 4: MAE = 0.04216
Fold 5: MAE = 0.04157
RandomForest        : MAE = 0.04205 ± 0.00185
Fold 1: MAE = 0.03407
Fold 2: MAE = 0.03929
Fold 3: MAE = 0.03729
Fold 4: MAE = 0.03679
Fold 5: MAE = 0.03548
ExtraTrees          : MAE = 0.03658 ± 0.00176
Fold 1: MAE = 0.04705
Fold 2: MAE = 0.04780
Fold 3: MAE = 0.04778
Fold 4: MAE = 0.05185
Fold 5: MAE = 0.04587
GradientBoosting    : MAE = 0.04807 ± 0.00202
Fold 1: MAE = 0.04051
Fold 2: MAE = 0.04194
Fold 3: MAE = 0.04280
Fold 4: MAE = 0.04381
Fold 5: MAE = 0.03689
HistGradientBoosting: MAE = 0.04119 ± 0.00241
Fold 1: MAE = 0.04585
Fold 2: MAE = 0.04443
Fold 3: MAE = 0.04376
Fold 4: MAE = 0.03830
Fold 5: MAE = 0.03709
XGBoost             : MAE = 0.04188 ± 0.00351
Fold 1: MAE = 0.04191
Fold 2: MAE = 0.04309
Fold 3: MAE = 0.04205
Fold 4: MAE = 0.04357
Fold 5: MAE = 0.03849
LightGBM            : MAE = 0.04182 ± 0.00178
Fold 1: MAE = 0.03898
Fold 2: MAE = 0.03968
Fold 3: MAE = 0.0381

## Rg

In [25]:
models_to_compare = {
    'RandomForest': lambda: RandomForestRegressor(),
    'ExtraTrees': lambda: ExtraTreesRegressor(),
    'GradientBoosting': lambda: GradientBoostingRegressor(),
    'HistGradientBoosting': lambda: HistGradientBoostingRegressor(),
    'XGBoost': lambda: XGBRegressor(verbosity=0),
    'LightGBM': lambda: LGBMRegressor(verbose=-1),
    'CatBoost': lambda: CatBoostRegressor(verbose=0),
    'Ridge': lambda: Ridge(),
    'Lasso': lambda: Lasso(),
    'ElasticNet': lambda: ElasticNet(),
    'Huber': lambda: HuberRegressor(),
    'KNN': lambda: KNeighborsRegressor(),
    'SVR': lambda: SVR(),
    'AdaBoost': lambda: AdaBoostRegressor(),
}

for name, model_cls in models_to_compare.items():
    try:
        mean_mae, std_mae = model_cv(rg, model_cls, 'Rg', n_splits=5)
        print(f'{name:<20}: MAE = {mean_mae:.5f} ± {std_mae:.5f}')
    except Exception as e:
        print(f'{name:<20}: Failed with error: {e}')

Fold 1: MAE = 1.93715
Fold 2: MAE = 1.89352
Fold 3: MAE = 2.07612
Fold 4: MAE = 1.50010
Fold 5: MAE = 1.72785
RandomForest        : MAE = 1.82695 ± 0.19764
Fold 1: MAE = 1.93537
Fold 2: MAE = 1.86332
Fold 3: MAE = 1.95463
Fold 4: MAE = 1.45443
Fold 5: MAE = 1.68835
ExtraTrees          : MAE = 1.77922 ± 0.18760
Fold 1: MAE = 2.03793
Fold 2: MAE = 1.86112
Fold 3: MAE = 2.02143
Fold 4: MAE = 1.51595
Fold 5: MAE = 1.80560
GradientBoosting    : MAE = 1.84841 ± 0.18888
Fold 1: MAE = 2.11304
Fold 2: MAE = 1.89096
Fold 3: MAE = 2.01859
Fold 4: MAE = 1.59132
Fold 5: MAE = 1.76936
HistGradientBoosting: MAE = 1.87665 ± 0.18393
Fold 1: MAE = 2.08787
Fold 2: MAE = 1.91450
Fold 3: MAE = 2.12773
Fold 4: MAE = 1.58404
Fold 5: MAE = 1.62381
XGBoost             : MAE = 1.86759 ± 0.22725
Fold 1: MAE = 2.06008
Fold 2: MAE = 1.84151
Fold 3: MAE = 1.99887
Fold 4: MAE = 1.63982
Fold 5: MAE = 1.78186
LightGBM            : MAE = 1.86443 ± 0.15109
Fold 1: MAE = 1.94091
Fold 2: MAE = 1.82176
Fold 3: MAE = 1.8671

# Final Model For Submission

## 📘 Model Training & Prediction with Cross-Validation

This notebook defines a reusable training and evaluation pipeline using **K-Fold Cross-Validation**, and applies it to predict multiple polymer properties using different machine learning models.

---

### 🧠 Cross-Validation Function

```python
def model_cv_predict(train_d, test_d, model_fn, target, n_splits=5):

In [26]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from catboost import CatBoostRegressor

# クロスバリデーション付きモデル予測関数
def model_cv_predict(train_d, test_d, model_fn, target, n_splits=5):
    X = train_d.drop(columns=[target])
    y = train_d[target].copy()
    test_preds = np.zeros(len(test_d))
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    maes = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        m = model_fn()
        m.fit(X_train, y_train)

        val_pred = m.predict(X_val)
        mae = mean_absolute_error(y_val, val_pred)
        maes.append(mae)

        test_preds += m.predict(test_d) / n_splits
        print(f"Fold {fold}: MAE = {mae:.5f}")

    print(f"CV MAE = {np.mean(maes):.5f} ± {np.std(maes):.5f}")
    return test_preds

# submission 用予測
sub = {
    'id': ID,
    'Tg': model_cv_predict(tg, test, lambda: CatBoostRegressor(verbose=0), 'Tg'),
    'FFV': model_cv_predict(ffv, test, lambda: ExtraTreesRegressor(), 'FFV'),
    'Tc': model_cv_predict(tc, test, lambda: CatBoostRegressor(verbose=0), 'Tc'),
    'Density': model_cv_predict(density, test, lambda: ExtraTreesRegressor(), 'Density'),
    'Rg': model_cv_predict(rg, test, lambda: ExtraTreesRegressor(), 'Rg')
}

# CSVとして保存
submission_df = pd.DataFrame(sub)
submission_df.to_csv("submission.csv", index=False)
print("✅ submission.csv を保存しました")

Fold 1: MAE = 37.35201
Fold 2: MAE = 34.47226
Fold 3: MAE = 36.68867
Fold 4: MAE = 39.82375
Fold 5: MAE = 38.58171
CV MAE = 37.38368 ± 1.80829
Fold 1: MAE = 0.00614
Fold 2: MAE = 0.00612
Fold 3: MAE = 0.00608
Fold 4: MAE = 0.00603
Fold 5: MAE = 0.00646
CV MAE = 0.00617 ± 0.00015
Fold 1: MAE = 0.02858
Fold 2: MAE = 0.02848
Fold 3: MAE = 0.03989
Fold 4: MAE = 0.02996
Fold 5: MAE = 0.03540
CV MAE = 0.03246 ± 0.00450
Fold 1: MAE = 0.03459
Fold 2: MAE = 0.03900
Fold 3: MAE = 0.03601
Fold 4: MAE = 0.03827
Fold 5: MAE = 0.03675
CV MAE = 0.03692 ± 0.00158
Fold 1: MAE = 1.94000
Fold 2: MAE = 1.85362
Fold 3: MAE = 1.94768
Fold 4: MAE = 1.42098
Fold 5: MAE = 1.67974
CV MAE = 1.76840 ± 0.19870
✅ submission.csv を保存しました


In [27]:
submission_df

,id,Tg,FFV,Tc,Density,Rg
0,1109053969,165.076659,0.373235,0.211291,1.162820,20.743923
1,1422188626,193.840622,0.374802,0.246583,1.109561,20.735156
2,2032016830,125.141618,0.350523,0.257867,1.087157,20.857618
